In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
with open('../data/main_test.pkl', 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.s1_preprocessing import s1_amp_to_dB, s1_dB_to_amp, merge_partial_s1_images

ds = ds.isel(time = slice(0, 6))

In [ ]:
from rioxarray.merge import merge_arrays

def merge_s1_times(dataset, times, verbose = False):
    if verbose:
        print('Merging:', times)
    das = [dataset.sel(time = ts)['s1'] for ts in times]
    assert len(das) == len([d for d in das if d['relative_orbit'] == das[0]['relative_orbit']])
    print(das[0].where(das[0] == 0).notnull().sum())
    if das[0].where(das[0] == 0).notnull().sum() > 0:
        nodata_value = 0
    else:
        nodata_value = np.nan
    print(nodata_value)
    merged = merge_arrays(das, crs = 'EPSG:4326', nodata= nodata_value)
    dataset = dataset.drop_sel(time = times[1:])
    dataset['s1'].loc[dict(time = times[0])] = merged.values
    times = []
    return dataset, times, das

import pandas as pd

def s1_partial_merge(dataset):
    times = []
    for ts in dataset.time.values:
        
        if not times:
            times.append(ts)
            continue

        if ts - times[0] > pd.Timedelta('1 minute'):
            if len(times) == 1:
                times = [ts]
                continue
            else:
                dataset, times, das = merge_s1_times(dataset, times)
        else:
            times.append(ts)
        
        if ts == ds.time.values[-1] and len(times) > 1:
            dataset, times, das = merge_s1_times(dataset, times)
        
    return dataset, das

In [ ]:
s1_amp_to_dB(ds)

In [ ]:
ds1, das = s1_partial_merge(ds)

In [ ]:
ds1.isel(time = 0).sel(band = 'VV')['s1'].plot()

In [ ]:
ds.isel(time = 0).sel(band = 'VV')['s1']

In [ ]:
ds1.isel(time = 0).sel(band = 'VV')['s1'].plot()

In [ ]:
s1_amp_to_dB(ds)

In [ ]:
ds.isel(time = 0).sel(band = 'VV')['s1']

In [ ]:
ds.isel(time = 0)['time'].values

In [ ]:
ds1 = s1_partial_merge(ds)

In [ ]:
ds1['s1'].isel(time = 0).sel(band = 'VV').plot(vmax = 0.5)

In [ ]:
da1 = ds['s1'].isel(time = 0).sel(band = 'VV')

da2 = ds['s1'].isel(time = 1).sel(band = 'VV')


In [ ]:
da1.plot(vmax = 0.5)

In [ ]:
da2.plot(vmax = 0.5)

In [ ]:
from rioxarray.merge import merge_arrays
merged = merge_arrays([da1, da2], crs = 'EPSG:4326', nodata = np.nan)
merged.plot(vmax = 0.5)

In [ ]:
da1.plot(vmax = 0.5)

In [ ]:
# plot first time slice and VV band to see initial
ds['s1'].isel(time = 1).sel(band = 'VV').plot()
plt.show()
# snag the original time slice to compare afterwards
amp = ds['s1'].isel(time = 1).sel(band = 'VV').values.ravel()
amp = amp[amp != 0]
# run your newly written function over the dataset and capture returning dataset
s1_amp_to_dB(ds)
# plot first time slice and VV band to see dB change
ds['s1'].isel(time = 1).sel(band = 'VV').plot()
plt.show()
# grab the first time slice VV
dB = ds['s1'].isel(time = 1).sel(band = 'VV').values.ravel()
dB = dB[~np.isnan(dB)]
# assert the conversion was succesful
assert np.allclose(dB, 10 * np.log10(amp))

In [ ]:
ds['s1'].isel(time = 0).sel(band = 'VV')[2000, 2000] = 0

In [ ]:
ds['s1'].isel(time = 0).sel(band = 'VV')[2000, 2000]

In [ ]:
ds['s1'].isel(time = 1, band = 0, x = 10, y = 10)

In [ ]:
# plot first time slice and VV band to see initial
ds['s1'].isel(time = 1).sel(band = 'VV').plot()
plt.show()
# snag the original time slice to compare afterwards
dB = ds['s1'].isel(time = 1).sel(band = 'VV').values.ravel()
# run your newly written function over the dataset and capture returning dataset
s1_dB_to_amp(ds)
# plot first time slice and VV band to see dB change
ds['s1'].isel(time = 1).sel(band = 'VV').plot()
plt.show()
# grab the first time slice VV
dB = ds['s1'].isel(time = 1).sel(band = 'VV').values.ravel()
dB = dB[np.isfinite(dB)]
# assert the conversion was succesful
assert np.allclose(dB, 10 * np.log10(amp))

In [ ]:
ds = ds.set_xindex('relative_orbit')

In [ ]:
ds.sel(relative_orbit = 20)

In [ ]:
ds.where(ds.flight_dir == 'ascending', drop = True)